# Домашнее задание 4

In [1]:
import numpy as np
import pandas as pd
import random

import os
import importlib

from kaggle_environments import make, evaluate

No pygame installed, ignoring import


## Подготовка агентов

In [2]:
%%writefile rock_agent.py

def rock_agent(observation, configuration):
    # Агент, который всегда выбирает камень
    return 0

Overwriting rock_agent.py


In [3]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    # Агент, который всегда выбирает бумагу
    return 1

Overwriting paper_agent.py


In [4]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    # Агент, который всегда выбирает ножницы
    return 2

Overwriting scissors_agent.py


In [5]:
%%writefile counter_attacker_agent.py

import random

def counter_attacker_agent(observation, configuration):
    # Агент, который контр-атакует предыдущий выбор соперника 
    if observation.step == 0:
        # При первом раунде выбирает случайный жест
        return random.randrange(0, configuration.signs)
    else:
        # При последующих контр-атакует предыдущий выбор соперника 
        return (observation.lastOpponentAction + 1) % 3

Overwriting counter_attacker_agent.py


In [6]:
%%writefile opposite_counter_attacker_agent.py

import random

def opposite_counter_attacker_agent(observation, configuration):
    # Агент, который выбирает жест, проигрывающий предыдущему выбору соперника
    if observation.step == 0:
        # При первом раунде выбирает случайный жест
        return random.randrange(0, configuration.signs)
    else:
        # При последующих выбирает жест, проигрывающий предыдущему выбору соперника
        return (observation.lastOpponentAction - 1) % 3

Overwriting opposite_counter_attacker_agent.py


In [7]:
%%writefile mirror_agent.py

import random

def mirror_agent(observation, configuration):
    # Агент, который копирует предыдущий выбор соперника
    if observation.step == 0:
        # При первом раунде выбирает случайный жест
        return random.randrange(0, configuration.signs)
    else:
        # При последующих копирует предыдущий выбор соперника
        return observation.lastOpponentAction

Overwriting mirror_agent.py


In [8]:
%%writefile opposite_mirror_agent.py

import random

def opposite_mirror_agent(observation, configuration):
    # Агент, который выбирает жест, который отличается от предыдущего выбора соперника
    if observation.step == 0:
        # При первом раунде выбирает случайный жест
        return random.randrange(0, configuration.signs)
    else:
        # При последующих выбирает жест, который отличается от предыдущего выбора соперника
        options = [move for move in range(configuration.signs) if move != observation.lastOpponentAction]
        return random.choice(options)

Overwriting opposite_mirror_agent.py


In [9]:
%%writefile cycle_agent.py

def cycle_agent(observation, configuration):
    # Агент, который выбирает все жесты по порядку
    return observation.step % 3

Overwriting cycle_agent.py


In [10]:
%%writefile opposite_win_agent.py

import random

def opposite_win_agent(observation, configuration):
    # Агент, который меняет жест после победы
    if not hasattr(opposite_win_agent, "last_reward"):
        # Инициализация атрибутов функции для хранения последнего вознаграждения и выигрыша
        opposite_win_agent.last_reward = 0
        opposite_win_agent.last_action = random.randrange(0, configuration.signs)
    
    # Определение успешности розыгрыша, как разницы между суммой очков текущего и предыдущего раундов
    last_result = observation.reward - opposite_win_agent.last_reward
    opposite_win_agent.last_reward = observation.reward 

    if last_result > 0:
        # Выбирает любой другой жест в случае выйгрыша
        next_move = random.choice([move for move
                                   in range(configuration.signs)
                                   if move != opposite_win_agent.last_action])
    else:
        # Выбирает случайный жест при проигрыше или ничьей в предыдущем раунде
        next_move = random.randrange(0, configuration.signs)

    # Обновление предыдущего раунда
    opposite_win_agent.last_action = next_move
    return next_move

Overwriting opposite_win_agent.py


In [11]:
%%writefile counter_attacker_win_agent.py

import random

def counter_attacker_win_agent(observation, configuration):
    # Агент, который контр-атакует после победы
    if not hasattr(counter_attacker_win_agent, "last_reward"):
        # Инициализация атрибутов функции для хранения последнего вознаграждения и выигрыша
        counter_attacker_win_agent.last_reward = 0
        counter_attacker_win_agent.last_action = random.randrange(0, configuration.signs)
    
    # Определение успешности розыгрыша, как разницы между суммой очков текущего и предыдущего раундов
    last_result = observation.reward - counter_attacker_win_agent.last_reward
    counter_attacker_win_agent.last_reward = observation.reward 

    if last_result > 0:
        # Контр-атакует в случае выйгрыша
        next_move = (counter_attacker_win_agent.last_action + 1) % 3
    else:
        # Выбирает случайный жест при проигрыше или ничьей в предыдущем раунде
        next_move = random.randrange(0, configuration.signs)

    # Обновление предыдущего раунда
    counter_attacker_win_agent.last_action = next_move
    return next_move

Overwriting counter_attacker_win_agent.py


In [12]:
%%writefile opposite_lose_agent.py

import random

def opposite_lose_agent(observation, configuration):
    # Агент, который меняет жест после поражения
    if not hasattr(opposite_lose_agent, "last_reward"):
        # Инициализация атрибутов функции для хранения последнего вознаграждения и выигрыша
        opposite_lose_agent.last_reward = 0
        opposite_lose_agent.last_action = random.randrange(0, configuration.signs)
    
    # Определение успешности розыгрыша, как разницы между суммой очков текущего и предыдущего раундов
    last_result = observation.reward - opposite_lose_agent.last_reward
    opposite_lose_agent.last_reward = observation.reward 

    if last_result < 0:
        # Выбирает любой другой жест в случае проигрыша
        next_move = random.choice([move for move
                                   in range(configuration.signs)
                                   if move != opposite_lose_agent.last_action])
    else:
        # Выбирает случайный жест при проигрыше или ничьей в предыдущем раунде
        next_move = random.randrange(0, configuration.signs)

    # Обновление предыдущего раунда
    opposite_lose_agent.last_action = next_move
    return next_move

Overwriting opposite_lose_agent.py


In [13]:
%%writefile counter_attacker_lose_agent.py

import random

def counter_attacker_lose_agent(observation, configuration):
    # Агент, который меняет жест после поражения
    if not hasattr(counter_attacker_lose_agent, "last_reward"):
        # Инициализация атрибутов функции для хранения последнего вознаграждения и выигрыша
        counter_attacker_lose_agent.last_reward = 0
        counter_attacker_lose_agent.last_action = random.randrange(0, configuration.signs)
    
    # Определение успешности розыгрыша, как разницы между суммой очков текущего и предыдущего раундов
    last_result = observation.reward - counter_attacker_lose_agent.last_reward
    counter_attacker_lose_agent.last_reward = observation.reward 

    if last_result < 0:
        # Выбирает любой другой жест в случае проигрыша
        next_move = (counter_attacker_lose_agent.last_action + 1) % 3
    else:
        # Выбирает случайный жест при проигрыше или ничьей в предыдущем раунде
        next_move = random.randrange(0, configuration.signs)

    # Обновление предыдущего раунда
    counter_attacker_lose_agent.last_action = next_move
    return next_move

Overwriting counter_attacker_lose_agent.py


In [14]:
%%writefile random_agent.py

import random

def random_agent(observation, configuration):
    # Всегда выбирает случайный жест
    return random.randrange(0, configuration.signs)

Overwriting random_agent.py


In [15]:
%%writefile frequency_agent.py

import random

def frequency_agent(observation, configuration):
    # Агент,который выбирает наиболее используемый жест соперника
    if not hasattr(frequency_agent, "signs") or observation.step == 0:
        # Инициализация атрибута для хранения кол-ва использований жестов
        frequency_agent.signs = {0: 0, 1: 0, 2: 0}
        
    if observation.step == 0:
        # При первом раунде выбирает случайный жест
        return random.randrange(0, configuration.signs)
    else:
        # В последующих раундах выбирает наиболее используемый жест соперника
        frequency_agent.signs[observation.lastOpponentAction] += 1
        return (max(frequency_agent.signs, key=frequency_agent.signs.get) + 1) % 3
    

Overwriting frequency_agent.py


In [16]:
%%writefile opposite_frequent_agent.py

import random

def opposite_frequent_agent(observation, configuration):
    # Агент,который выбирает наименее используемый жест соперника
    if not hasattr(opposite_frequent_agent, "signs") or observation.step == 0:
        # Инициализация атрибута для хранения кол-ва использований жестов
        opposite_frequent_agent.signs = {0: 0, 1: 0, 2: 0}
        
    if observation.step == 0:
        # При первом раунде выбирает случайный жест
        return random.randrange(0, configuration.signs)    
    
    else:
        # В последующих раундах выбирает наиболее используемый жест соперника
        opposite_frequent_agent.signs[observation.lastOpponentAction] += 1
        return (min(opposite_frequent_agent.signs, key=opposite_frequent_agent.signs.get) + 1) % 3

Overwriting opposite_frequent_agent.py


In [17]:
%%writefile scissor_first.py

import random

def scissor_first(observation, configuration):
    # Агент, который всегда начинает с ножниц
    if observation.step == 0:
        # В первом раунде выбирает ножницы
        return 2
    else:
        # В последующих раундах выбирает случайный жест
        return random.randrange(0, configuration.signs)

Overwriting scissor_first.py


## Проведение соревнований

In [18]:
# Формирования списка с названиями агентов
agents_list = [file for file in os.listdir('.') if file.endswith('.py')]
agents_list = [agent[:-3] for agent in agents_list]

# Словарь для хранения функций агентов
agents = {}

# Импорт функций агентов
for agent in agents_list:
    module = importlib.import_module(agent)
    agents[agent] = getattr(module, agent)

# Словарь для хранения результатов
scores = {agent_name: 0 for agent_name in agents.keys()}

# Турнир
for i, agent_name_1 in enumerate(agents.keys()):
    for j, agent_name_2 in enumerate(agents.keys()):
        if i != j:
            env = make("rps", configuration={"episodeSteps": 100})
            env.reset()
            result = env.run([agents[agent_name_1], agents[agent_name_2]])
            
            
            # Подсчет очков
            scores[agent_name_1] += result[-1][0]["reward"]
            scores[agent_name_2] += result[-1][1]["reward"]
        else:
            continue

# Сортировка и демонстрация результатов
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for agent, score in sorted_scores:
    print(f"{agent}: {score}")

frequency_agent: 969.0
counter_attacker_agent: 575.0
counter_attacker_win_agent: 360.0
opposite_mirror_agent: 98.0
scissors_agent: 52.0
rock_agent: 26.0
random_agent: 0
scissor_first: 0
opposite_win_agent: -23.0
opposite_lose_agent: -39.0
opposite_counter_attacker_agent: -202.0
paper_agent: -221.0
opposite_frequent_agent: -277.0
cycle_agent: -289.0
counter_attacker_lose_agent: -514.0
mirror_agent: -515.0
